In [27]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pyro import clear_param_store
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import torch
import arviz as az
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral
from bokeh.models import Slope, Div
from bokeh.io import curdoc,output_notebook
from bokeh.layouts import column,gridplot
from sklearn.metrics import r2_score
from seaborn import clustermap
from bokeh.models import Band, ColumnDataSource
from scipy import stats
from sklearn.metrics import root_mean_squared_error

In [28]:
train=pd.read_excel(r"../data/chem_train.xlsx")


In [29]:
test=pd.read_excel(r"../data/chem_test.xlsx")


In [30]:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,examine,help"

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
X_form=torch.tensor(train.corrected_week.values).float().to(device)
y_form=torch.tensor(train.formaldehyd.values).float().to(device)

In [33]:
X_form

tensor([ 37.3198,  50.8112,  22.2611,  ..., -17.3862, -16.8562, -15.9360],
       device='cuda:0')

In [34]:
clear_param_store()
rbf = gp.kernels.RBF(input_dim=1)


rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(20.)))

rbf.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(10.)))

gpr = gp.models.GPRegression(X_form,y_form, rbf).to(device)
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(10.)))


nuts_kernel = NUTS(gpr.model,target_accept_prob=0.9)


mcmc = MCMC(nuts_kernel,warmup_steps=9000, num_samples=1000,num_chains=1 )

mcmc.run()

Sample: 100%|██████████████████████████████████████| 10000/10000 [08:55, 18.68it/s, step size=6.35e-01, acc. prob=0.941]


In [35]:
torch.save(gpr, "../models/form_seasonality");

In [36]:
posterior_samples = mcmc.get_samples(500)
posterior_predictive= Predictive(gpr, posterior_samples)(X_form)
prior = Predictive(gpr, num_samples=500)(X_form)

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive,

)
az.to_json(pyro_data, "../Arviz_stats/mcmc_form_seasonality.json")

/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_form_seasonality.json'

In [37]:
X_acetald=torch.tensor(train.corrected_week.values).float().to(device)
y_acetald=torch.tensor(train.acetald.values).float().to(device)

In [38]:
clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)


rbf_bc.variance = PyroSample(dist.HalfNormal(torch.tensor(20.)))

rbf_bc.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(10.)))
gpr_bc = gp.models.GPRegression(X_acetald,y_acetald, rbf_bc).to(device)
gpr_bc.noise = PyroSample(dist.HalfNormal(torch.tensor(10.)))


nuts_kernel_bc = NUTS(gpr_bc.model,target_accept_prob=0.9)


mcmc_bc = MCMC(nuts_kernel_bc,warmup_steps=9000, num_samples=1000,num_chains=1)

mcmc_bc.run()

Sample: 100%|██████████████████████████████████████| 10000/10000 [09:14, 18.05it/s, step size=6.14e-01, acc. prob=0.935]


In [39]:
torch.save(gpr_bc, "../models/acetald_seasonality");

In [40]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(X_acetald)
prior_bc = Predictive(gpr_bc, num_samples=500)(X_acetald)

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
az.to_json(pyro_data_bc, "../Arviz_stats/mcmc_acetald_seasonality.json")

'../Arviz_stats/mcmc_acetald_seasonality.json'

In [41]:
1

1

In [42]:
X_acetone=torch.tensor(train.corrected_week.values).float().to(device)
y_acetone=torch.tensor(train.acetone.values).float().to(device)

In [43]:
clear_param_store()
rbf_ac = gp.kernels.RBF(input_dim=1)


rbf_ac.variance = PyroSample(dist.HalfNormal(torch.tensor(20.)))

rbf_ac.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(10.)))

gpr_ac = gp.models.GPRegression(X_acetone,y_acetone, rbf_ac).to(device)
gpr_ac.noise = PyroSample(dist.HalfNormal(torch.tensor(10.)))


nuts_kernel_ac = NUTS(gpr_ac.model,target_accept_prob=0.9)


mcmc_ac = MCMC(nuts_kernel_ac,warmup_steps=9000, num_samples=1000,num_chains=1)

mcmc_ac.run()

Sample: 100%|██████████████████████████████████████| 10000/10000 [07:56, 20.97it/s, step size=3.82e-01, acc. prob=0.968]


In [44]:
torch.save(gpr_ac, "../models/acetone_seasonality");

In [45]:
posterior_samples_ac = mcmc_ac.get_samples(500)
posterior_predictive_ac = Predictive(gpr_bc, posterior_samples_ac)(X_acetone)
prior_ac = Predictive(gpr_ac, num_samples=500)(X_acetone)

pyro_data_ac = az.from_pyro(mcmc_ac,
    prior=prior_ac,
    posterior_predictive=posterior_predictive_ac,

)
az.to_json(pyro_data_ac, "../Arviz_stats/mcmc_acetone_seasonality.json")

'../Arviz_stats/mcmc_acetone_seasonality.json'

In [46]:
form=gpr(torch.tensor(test.corrected_week.values).float().to(device))[0].cpu().detach().numpy()
acetald=gpr_bc(torch.tensor(test.corrected_week.values).float().to(device))[0].cpu().detach().numpy()
acetone=gpr_ac(torch.tensor(test.corrected_week.values).float().to(device))[0].cpu().detach().numpy()

In [47]:
r2_score(test.formaldehyd,form),r2_score(test.acetald,acetald),r2_score(test.acetone,acetone)

(0.023508300811006744, 0.029321839168464092, 0.14811447095572172)

In [48]:
root_mean_squared_error(test.formaldehyd,form),root_mean_squared_error(test.acetald,acetald),root_mean_squared_error(test.acetone,acetone)

(7.446443316062849, 4.266915306534569, 5.593722764445667)